In [ ]:
import os
import numpy as np
import tensorflow as tf
import h5py
from modules import models
from modules import utility as uty
from matplotlib import pyplot as plt
import pickle as pkl

uty.handle_GPUs(GPUs = '5', enable_GPU=1,)

## Hyper-Parameters

In [ ]:
# speed-up factor
R = 8.0

# decoder type (dec {0, 1, 2})
dec = 2

# loss type (L {0, 1, 2})
L = 0

# regularization weight
phi = 0

# batch_size used to train MRI dataset is 16
batch_size = 128

lr = 0.001

max_epochs_train = 10000

In [ ]:
assert ((dec==0 or dec==2) and L==0) or (dec==1 and (L==1 or L==2)), 'check for the right combinations of "dec" and "L"'

## Load Dataset

Models have been trained on the fastMRI dataset (https://fastmri.org/dataset/) Knee Emulated Single (esc) Coil Track

In [ ]:
dataset = 'PD'

trainDataPath = os.path.join('..','..','datasets','kneeMRI','fastMRI_trainSet_esc_320_'+dataset+'.hdf5') 
with h5py.File(trainDataPath, 'r') as f:
        xdata = f[os.path.join('dataset')] [()]
        
valDataPath = os.path.join('..','..','datasets','kneeMRI','fastMRI_valSet_esc_320_'+dataset+'.hdf5')
with h5py.File(valDataPath, 'r') as f:
        vdata = f[os.path.join('dataset')] [()]
        
testDataPath = os.path.join('..','..','datasets','kneeMRI','fastMRI_testSet_esc_320_'+dataset+'.hdf5')
with h5py.File(testDataPath, 'r') as f:
        tdata = f[os.path.join('dataset')] [()]

input_shape = np.shape(xdata[0]) 

print('dataset in use = ',dataset)
print('train dataset size: ',np.shape(xdata))
print('evaluation dataset size: ',np.shape(vdata))
print('test dataset size: ',np.shape(tdata))

# Load Model

The model class is loaded.  
The depth attribute determines the depth of the U-NET. Because MNIST
is a small dataset U-NET can coherentely be small and depth = 2 is used.  
Default value is depth = 5.

In [ ]:
model = uty.loupe_model(input_shape, R, dec, L, depth = 5,)

In [ ]:
path_to_save = os.path.join('save_model','dec'+str(dec)+'-L'+str(L)+'-R'+str(R))
model.load_weights(path_to_save)

In [ ]:
model.summary()

## Compile

Mean Absolute Error (mae) is used to compute the error between the
spatial images.   
Mean Square Error (mse) is used to compute the error between the spatial arrays.

Peak Signal to Noise Ration (PSNR) and Structural Similarity (SSIM) are not used 
as training losses but as additional monitors.

Dec0 and Dec2 only takes one output (the reconstructed image),
Dec1 takes two (the reconstructed image and 
the difference between the senses k-space and the encoded reconstructed image). 


In [ ]:
def metric_PSNR(y_true, y_pred):
    return tf.reduce_mean(tf.image.psnr(y_true, y_pred, 1))

def metric_SSIM(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1))

# regularization term used in L = {1, 2}
def loss_norm_y(_, y_pred):
    return tf.reduce_mean(tf.norm(y_pred))

In [ ]:
if dec == 0 or dec == 2:
    loss = 'mae'
    loss_weights = [1]
    metrics = [[metric_PSNR, metric_SSIM]]

elif dec == 1:
    loss = ['mae', loss_norm_y]
    loss_weights = [1-phi, phi]
    metrics = [[metric_PSNR, metric_SSIM], []]
    
    
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
              loss = loss,
              loss_weights = loss_weights,
              metrics = metrics,
             )

## Evaluation

Before running the tests, it is necessary to 
1. fix, and
2. binarize  

the mask.   
This is necessary because during training the mask is randomly generated 
in accordance with a trainable probability mask.   
This probability mask, to be trainable has to 
generate continuous values (not binary).


We do this by calling "change_setting( ..., setting = 'test')"  
By calling the same function with " setting = 'train' " the mask is unfixed and 
un-binarized.



In [ ]:
setting = 'test'
verbose_settings = False

model = uty.change_setting(model, setting = setting, 
                           verbose = verbose_settings)

In [ ]:
model.evaluate(tdata, tdata, batch_size = 1);

## Dykstra

Copy from the paper!

In [ ]:
iterations_Dykstra = 50

model_Dykstra = models.add_Dykstra_projection_to_model(
        model, 
        iterations = iterations_Dykstra)

In [ ]:
setting = 'test'
verbose_settings = False

model_Dykstra = uty.change_setting(model_Dykstra, 
                                   setting = setting, 
                                   verbose = verbose_settings)

model_Dykstra.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
                      loss = loss,
                      loss_weights = loss_weights,
                      metrics = metrics,
                     );

dopo avere aggiunto il 

In [ ]:
model_Dykstra.evaluate(tdata, tdata, batch_size = 1);

## Mask

In [ ]:
prob_mask = np.array(uty.read_probMask(model))[0,...,0]

mask = prob_mask > 0.5
mask_rot = uty._rotate_corners_mask(mask, plot = True, );

## Reconstructions

In [ ]:
mri = tdata[30][np.newaxis]

model_encoded = tf.keras.Model(inputs = model.inputs, 
                               outputs = model.get_layer('ifft').output, )

mri_encoded = np.array(model_encoded.predict(mri))

mri_decoded = model.predict(mri)

mri_Dykstra = np.array(model_Dykstra.predict(mri))

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_figheight(10)
fig.set_figwidth(12)
ax[0].imshow(mri[0, ..., 0], vmin = 0, vmax  = 1, cmap = 'gray')
ax[0].title.set_text('ground truth')
ax[1].imshow(mri_encoded[0, ..., 0], vmin = 0, vmax  = 1, cmap = 'gray')
ax[1].title.set_text('encoded')
ax[2].imshow(mri_Dykstra[0, ..., 0], vmin = 0, vmax  = 1, cmap = 'gray')
ax[2].title.set_text('decoded')

# More utilities (see utility.py)

Those are some functions in the utility.py that may be useful to the user:

1. > uty.set_neurons_trainability(model, trainable, verbose)

freezes (or unfreezes) the DECODER

2. > uty.set_probMask_trainability(model, trainable, verbose)

freezes (or unfreezes) the ENCODER (mask generator)

3. > uty.set_slope_trainability(model, trainable, verbose)

sets the trainability of the slope "s" of the mask generator

4. > uty.set_mask_slope(model, slope, verbose)

sets the value of the slope "s" of the mask generator

5. > uty.set_mask_R(model, R, verbose)

sets the acceleration factor "R" of the mask generator

6. > uty.write_probMask(model, probMask, verbose)

replaces the prob mask with a user-defined one

7. > uty.copy_weights_by_matching_layer_name(model_dest, model, verbose)

given two similar models (imagine "model_dest" has few layers adjointed to/removed from "model"), all the layers sharing the same name will share the same weights after the function is used

8. > uty.handle_GPUs(GPUs, enable_GPU)

selects the GPUs to use and activates the memory growth (for memory efficient training)